# Part 1: Getting started

In [6]:
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
%matplotlib inline
seed = 0
np.random.seed(seed)
import tensorflow as tf
tf.random.set_seed(seed)
import os
os.environ['PATH'] = '/opt/Xilinx/Vivado/2020.1/bin:' + os.environ['PATH']
tf.config.threading.set_inter_op_parallelism_threads(8)
os.environ['OMP_NUM_THREADS'] = '1'

import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from callbacks import all_callbacks

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1

from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu
import tensorflow.compat.v1 as tf1

In [7]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

## Fetch the redwine dataset

In [8]:
df = pd.read_csv('./winequality-red.csv', sep = ';')
print (np.shape(df))
df

(1599, 12)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [9]:
X = df.drop('quality', axis = 1).values
y = df.quality
print(y)

0       5
1       5
2       5
3       6
4       5
       ..
1594    5
1595    6
1596    6
1597    5
1598    6
Name: quality, Length: 1599, dtype: int64


In [10]:
y[1:]

1       5
2       5
3       6
4       5
5       5
       ..
1594    5
1595    6
1596    6
1597    5
1598    6
Name: quality, Length: 1598, dtype: int64

In [11]:
le = LabelEncoder()
print(y)
y = le.fit_transform(y)
print(le.classes_)
y = to_categorical(y, 6)
print(y)

0       5
1       5
2       5
3       6
4       5
       ..
1594    5
1595    6
1596    6
1597    5
1598    6
Name: quality, Length: 1599, dtype: int64
[3 4 5 6 7 8]
[[0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 ...
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]


In [12]:
print(y[1])

[0. 0. 1. 0. 0. 0.]


In [13]:
print(X)
print(y)

[[ 7.4    0.7    0.    ...  3.51   0.56   9.4  ]
 [ 7.8    0.88   0.    ...  3.2    0.68   9.8  ]
 [ 7.8    0.76   0.04  ...  3.26   0.65   9.8  ]
 ...
 [ 6.3    0.51   0.13  ...  3.42   0.75  11.   ]
 [ 5.9    0.645  0.12  ...  3.57   0.71  10.2  ]
 [ 6.     0.31   0.47  ...  3.39   0.66  11.   ]]
[[0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 ...
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]


In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [15]:
X_test

array([[ 7.7 ,  0.56,  0.08, ...,  3.24,  0.66,  9.6 ],
       [ 7.8 ,  0.5 ,  0.17, ...,  3.39,  0.48,  9.5 ],
       [10.7 ,  0.67,  0.22, ...,  3.28,  0.98,  9.9 ],
       ...,
       [ 6.7 ,  0.46,  0.24, ...,  3.39,  0.6 , 10.6 ],
       [10.5 ,  0.51,  0.64, ...,  3.09,  0.66, 11.8 ],
       [ 9.9 ,  0.5 ,  0.24, ...,  3.34,  0.52, 10.  ]])

In [16]:
ls=np.argmax(Y_test,axis=1)+3
print(ls)
#np.savetxt('output_classes.dat',ls, fmt='%d')

[6 5 6 5 6 5 5 5 5 6 7 3 5 5 6 7 5 7 8 5 5 6 5 6 6 6 7 6 5 6 5 5 6 5 6 5 7
 5 4 6 5 5 7 5 5 6 7 6 5 6 5 5 5 7 6 6 6 5 5 5 5 7 5 6 6 5 6 5 6 5 6 4 6 6
 6 5 8 5 6 6 5 6 5 6 6 7 5 6 7 4 7 6 5 5 5 6 5 6 5 6 5 5 5 7 6 7 6 5 6 5 8
 5 6 5 6 7 6 6 5 6 6 6 6 6 6 6 7 6 5 5 6 5 5 5 6 5 5 5 5 6 7 6 8 5 5 5 6 6
 6 5 6 7 6 5 6 5 5 6 6 6 7 5 7 5 5 5 6 6 5 5 6 5 7 6 7 6 6 5 5 6 4 6 5 7 5
 5 4 5 7 6 5 6 6 7 6 5 5 6 5 7 5 6 6 5 7 5 5 5 6 7 7 5 5 6 6 7 6 5 6 6 6 6
 6 7 4 5 5 7 5 5 5 5 6 6 5 7 5 6 6 6 5 4 6 7 6 7 5 6 6 5 5 6 5 6 4 5 6 6 5
 6 6 5 5 6 7 7 6 5 6 6 5 6 5 6 5 5 5 6 6 6 7 5 5 6 5 7 5 6 4 6 6 8 6 5 5 6
 5 7 6 6 5 5 7 6 6 5 6 6 5 7 6 6 6 6 5 6 5 5 6 4 6 6 6 5 5 5 6 6 6 6 4 7 6
 6 6 5 6 7 5 5 6 7 5 5 6 5 6 5 6 5 5 6 5 6 6 6 5 6 4 5 4 5 5 6 5 6 6 5 5 5
 5 5 6 5 6 6 6 5 5 6 5 5 6 6 6 7 6 5 5 6 6 5 5 6 7 6 5 6 5 7 5 5 7 5 6 7 7
 6 6 5 6 6 7 6 5 7 6 6 6 5 5 5 5 5 6 5 5 5 7 6 7 6 4 5 7 5 5 5 6 6 6 6 6 5
 6 5 6 5 6 6 7 4 6 5 6 6 7 5 7 5 5 6 5 5 6 5 6 5 5 6 6 4 5 6 5 7 8 6 7 4]


In [17]:
print(X_test)

[[ 7.7   0.56  0.08 ...  3.24  0.66  9.6 ]
 [ 7.8   0.5   0.17 ...  3.39  0.48  9.5 ]
 [10.7   0.67  0.22 ...  3.28  0.98  9.9 ]
 ...
 [ 6.7   0.46  0.24 ...  3.39  0.6  10.6 ]
 [10.5   0.51  0.64 ...  3.09  0.66 11.8 ]
 [ 9.9   0.5   0.24 ...  3.34  0.52 10.  ]]


In [18]:
print("Shape of X_train: ",X_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_train: ",Y_train.shape)
print("Shape of y_test",Y_test.shape)

Shape of X_train:  (1119, 11)
Shape of X_test:  (480, 11)
Shape of y_train:  (1119, 6)
Shape of y_test (480, 6)


In [19]:
print(X_test[1])

[7.80e+00 5.00e-01 1.70e-01 1.60e+00 8.20e-02 2.10e+01 1.02e+02 9.96e-01
 3.39e+00 4.80e-01 9.50e+00]


In [21]:
import pickle
from joblib import load
loaded_model = load('./RedWine.MLP_clf.joblib')

/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.24.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator MLPClassifier from version 0.24.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [22]:
print(X_test)

[[ 7.7   0.56  0.08 ...  3.24  0.66  9.6 ]
 [ 7.8   0.5   0.17 ...  3.39  0.48  9.5 ]
 [10.7   0.67  0.22 ...  3.28  0.98  9.9 ]
 ...
 [ 6.7   0.46  0.24 ...  3.39  0.6  10.6 ]
 [10.5   0.51  0.64 ...  3.09  0.66 11.8 ]
 [ 9.9   0.5   0.24 ...  3.34  0.52 10.  ]]


In [23]:
print(Y_test)

[[0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 ...
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0.]]


In [24]:
print(ls)

[6 5 6 5 6 5 5 5 5 6 7 3 5 5 6 7 5 7 8 5 5 6 5 6 6 6 7 6 5 6 5 5 6 5 6 5 7
 5 4 6 5 5 7 5 5 6 7 6 5 6 5 5 5 7 6 6 6 5 5 5 5 7 5 6 6 5 6 5 6 5 6 4 6 6
 6 5 8 5 6 6 5 6 5 6 6 7 5 6 7 4 7 6 5 5 5 6 5 6 5 6 5 5 5 7 6 7 6 5 6 5 8
 5 6 5 6 7 6 6 5 6 6 6 6 6 6 6 7 6 5 5 6 5 5 5 6 5 5 5 5 6 7 6 8 5 5 5 6 6
 6 5 6 7 6 5 6 5 5 6 6 6 7 5 7 5 5 5 6 6 5 5 6 5 7 6 7 6 6 5 5 6 4 6 5 7 5
 5 4 5 7 6 5 6 6 7 6 5 5 6 5 7 5 6 6 5 7 5 5 5 6 7 7 5 5 6 6 7 6 5 6 6 6 6
 6 7 4 5 5 7 5 5 5 5 6 6 5 7 5 6 6 6 5 4 6 7 6 7 5 6 6 5 5 6 5 6 4 5 6 6 5
 6 6 5 5 6 7 7 6 5 6 6 5 6 5 6 5 5 5 6 6 6 7 5 5 6 5 7 5 6 4 6 6 8 6 5 5 6
 5 7 6 6 5 5 7 6 6 5 6 6 5 7 6 6 6 6 5 6 5 5 6 4 6 6 6 5 5 5 6 6 6 6 4 7 6
 6 6 5 6 7 5 5 6 7 5 5 6 5 6 5 6 5 5 6 5 6 6 6 5 6 4 5 4 5 5 6 5 6 6 5 5 5
 5 5 6 5 6 6 6 5 5 6 5 5 6 6 6 7 6 5 5 6 6 5 5 6 7 6 5 6 5 7 5 5 7 5 6 7 7
 6 6 5 6 6 7 6 5 7 6 6 6 5 5 5 5 5 6 5 5 5 7 6 7 6 4 5 7 5 5 5 6 6 6 6 6 5
 6 5 6 5 6 6 7 4 6 5 6 6 7 5 7 5 5 6 5 5 6 5 6 5 5 6 6 4 5 6 5 7 8 6 7 4]


In [25]:
w1=loaded_model.coefs_[0]
b1=loaded_model.intercepts_[0]
w2=loaded_model.coefs_[1]
b2=loaded_model.intercepts_[1]

In [26]:
wb1=[]
wb1.append(w1)
wb1.append(b1)

wb2=[]
wb2.append(w2)
wb2.append(b2)

In [27]:
loaded_model

MLPClassifier(beta_1=0.8868131348915925, beta_2=0.6379705684637716,
              epsilon=0.42956678369024237, hidden_layer_sizes=2,
              learning_rate='adaptive', max_iter=150,
              momentum=0.7701281241915822, solver='lbfgs',
              validation_fraction=0.7861884994650608)

### genetic algorithm to determine the relu size, weight size, bias size and sparsity
x1: relu_size
x2: weight_size
x3: bias_size
x4: sparsity

problem: max( blackbox(x1,x2,x3,x4)
         min ( x1*x2*x3)
       s.t
         2 < x1,x2,x3 < 8
         x4 belongs to [0.2, 0.3, 0.4, 0.5]

In [30]:
import blackbox as bb
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.core.problem import ElementwiseProblem
import area as ar

In [31]:
class MyProblem(ElementwiseProblem):

    def __init__(self,weightsbiases1,weightsbiases2,layer1,layer2,layer3,X_test,X_train,Y_test,Y_train):
        self.weightsbiases1=weightsbiases1
        self.weightsbiases2=weightsbiases2
        self.layer1=layer1
        self.layer2=layer2
        self.layer3=layer3
        self.X_test=X_test
        self.X_train=X_train
        self.Y_test=Y_test
        self.Y_train=Y_train
        #x[0]: relu_size
        #x[1]: weight size layer1
        #x[2]: bias size layer1
        #x[3]: weight size layer2
        #x[4]: bias size layer2
        #x[5]: pruning sparsity
        #x[6]: input size
        
        super().__init__(n_var=7,
                         n_obj=2,
                         n_ieq_constr=0,
                         xl=np.array([3,2,2,2,2,2,2]),
                         xu=np.array([8,7,7,7,7,5,4]),
                         vtype=int)

    def _evaluate(self, x, out, *args, **kwargs):
        accuracy, weights = bb.blackbox(self.weightsbiases1,self.weightsbiases2, x[0], x[1], x[2], x[3], x[4], x[5], x[6] ,self.layer1, self.layer2, self.layer3, self.X_test, self.Y_test, self.X_train, self.Y_train)
        f1 = 1- accuracy
        f2 = ar.area(weights,x[6],x[0],x[1],x[3],self.layer1,self.layer2,self.layer3)

        out["F"] = [f1, f2]


In [32]:
layer1=11
layer2=2
layer3=6
problem = MyProblem(wb1,wb2,layer1,layer2,layer3,X_test,X_train,Y_test,Y_train)

In [33]:
algorithm = NSGA2(
    pop_size=5,
    n_offsprings=5,
    sampling=IntegerRandomSampling(),
    eliminate_duplicates=True
)

In [34]:
from pymoo.termination import get_termination

#set the number of generations
termination = get_termination("n_gen",1)

In [35]:
from pymoo.optimize import minimize

with tf.device('/cpu:0'):
    res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               save_history=True,
               verbose=True)
X = res.X
F = res.F

TypeError: blackbox() missing 1 required positional argument: 'Y_train'

In [ ]:
#accuracy, weights = bb.blackbox(wb1,wb2, 6, 5, 5, 5, 5, 3, 4 ,layer1, layer2, layer3, X_test, Y_test, X_train, Y_train)

In [ ]:
weights

In [ ]:
#ar.area(weights,4,6,5,5,layer1,layer2,layer3)

In [ ]:
print(res.F)

In [ ]:
print(res.X)

#### Create a list of all the pareto solutions

In [ ]:
paretos=res.X

### plot all the solutions from the final generation

In [ ]:
from pymoo.visualization.scatter import Scatter
pop=res.pop
vals=pop.get("F")
plot = Scatter()
plot.add(problem.pareto_front(), plot_type="line", color="black", alpha=0.7)
plot.add(vals, facecolor="none", edgecolor="red")
plot.show()

### plot the pareto solutions from the final generation

In [ ]:
plot = Scatter()
plot.add(problem.pareto_front(), plot_type="line", color="black", alpha=0.7)
plot.add(res.F, facecolor="none", edgecolor="blue")
plot.show()

### define an optimal solution

In [ ]:
relusize_f=4
weight_size_f1=2
bias_size_f1=5
weight_size_f2=5
bias_size_f2=3
sparsity=4.16
input_s=4
norm = 2**input_s

In [ ]:
relusize_f=6
weight_size_f1=5
bias_size_f1=2
weight_size_f2=3
bias_size_f2=4
sparsity=3
input_s=3
norm = 2**input_s
relusizeint=1

In [ ]:
relusize_f=5
weight_size_f1=2
bias_size_f1=2
weight_size_f2=6
bias_size_f2=2
sparsity=3.87

## normalize the input based on the selected bitwidth

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
for i in range(0,len(X_train)):
    X_train[i] = [round(x*norm)/norm for x in X_train[i]]

for i in range(0,len(X_test)):
    X_test[i] = [round(x*norm)/norm for x in X_test[i]]

In [ ]:
layer_1=11
layer_2=2
layer_3=6

### train the model with the optimal solution's parameters and evaluate it

In [ ]:
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning

sparsity_val = float(sparsity / 10)
weight_bias_size=[ [ (weight_size_f1,1), (bias_size_f1,1) ], [ (weight_size_f2,1), (bias_size_f2,1) ] ]
relu_size=(relusize_f,relusizeint)

model = Sequential()
model.add(QDense(layer_2, input_shape=(layer_1,), name='fc1', kernel_quantizer=quantized_bits(weight_bias_size[0][0][0],0,alpha=1,use_stochastic_rounding=True),bias_quantizer=quantized_bits(weight_bias_size[0][1][0],0,alpha=1),
                kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)   ))
model.add(QActivation(activation=quantized_relu(relu_size[0],relu_size[1],use_stochastic_rounding=False), name='relu1'))
model.add(QDense(layer_3, name='output',
                kernel_quantizer=quantized_bits(weight_bias_size[1][0][0],0,alpha=1,use_stochastic_rounding=True), bias_quantizer=quantized_bits(weight_bias_size[1][1][0],0,alpha=1),
                kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001 ) ))
model.add(Activation(activation='softmax', name='softmax'))


pruning_params = {"pruning_schedule" : pruning_schedule.ConstantSparsity(sparsity_val, begin_step=2000, frequency=100)}
model = prune.prune_low_magnitude(model, **pruning_params)
model.layers[0].set_weights(wb1)
model.layers[2].set_weights(wb2)

In [ ]:
from qkeras.utils import model_save_quantized_weights
adam = Adam(lr=0.001)
model.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
callbacks= all_callbacks( outputDir = 'model_red_wine_classification_prune')
callbacks.callbacks.append(pruning_callbacks.UpdatePruningStep())
model.fit(X_train, Y_train, batch_size=1,
        epochs=14,validation_split=0.2, verbose=0, shuffle=True,
        callbacks = callbacks.callbacks);
model = strip_pruning(model)
model.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
model_save_quantized_weights(model, "test_weights")


accuracy=model.evaluate(X_test,Y_test)
#print(model.get_weights())
print(" ACCURACY IS "+str(accuracy[1]) )

In [ ]:
print(model.get_weights())

In [ ]:
relu_size

In [ ]:
#GZ
print("copy top.v, sim.Xtest, and sim.Ytest to the synopsys project ")
from qkeras.utils import model_save_quantized_weights
model_save_quantized_weights(model, "test_weights")

def aptype_to_size(ap):
    if "ap_int" in ap:
        s=ap.split('<')[1].split('>')[0]
        i=s
    else:
        s,i=ap.split('<')[1].split('>')[0].split(',')
    return (int(s),int(i))

#get weights
print(model.get_weights())
#scale weights
allweights=[t*2**(weight_bias_size[j//2][j%2][0]-weight_bias_size[j//2][j%2][1]) for j,t in enumerate(model.get_weights())]
#scale=[2**(weight_bias_size[j//2][j%2][0]-weight_bias_size[j//2][j%2][1]) for j,_ in enumerate(model.get_weights())]
#print(scale)
print(allweights)
#cast weights to integer & transpose
allweightsT=[wl.T.astype(int).tolist() for wl in allweights]
print(allweightsT)
weight_list=allweightsT[0::2]
bias_list=allweightsT[1::2]
print(weight_list)
print(bias_list)

#set params
import hls4ml
config = hls4ml.utils.config_from_keras_model(model, granularity='name')

last_layer="linear"
input_size = (input_s,0)
relu_size= aptype_to_size(config['LayerName']['relu1']['Precision']['result'])


weight_bias_size= [
    [aptype_to_size(config['LayerName']['fc1']['Precision']['weight']),aptype_to_size(config['LayerName']['fc1']['Precision']['bias'])],
    [aptype_to_size(config['LayerName']['output']['Precision']['weight']),aptype_to_size(config['LayerName']['output']['Precision']['bias'])]
]  
sum_relu_size=[
    [(16,6),relu_size],
    [(16,6),(32,6)]
]
print(weight_bias_size)
print(sum_relu_size)


f=open("top.v","w")

import write_mlp_mergemult_ps as wv
wv.write_mlp_verilog(f, input_size, bias_list, weight_list, weight_bias_size, sum_relu_size,last_layer)
f.close()

f=open("sim.Xtest","w")
np.savetxt(f,(X_test*2**input_size[0]).astype(int),fmt='%d',delimiter=' ')
f.close()

from joblib import dump
dump(np.argmax(Y_test,axis=1), "sim.Ytest")

print("copy top.v, sim.Xtest, and sim.Ytest to the synopsys project ")


### Clustering

In [ ]:
w1=model.get_weights()

In [ ]:
model.set_weights(w1)

### define the network's architecture

In [ ]:
import clustering as cl
from importlib import reload
layer1=11
layer2=2 
layer3=6  

#### weight clustering for the first layer

In [ ]:
target_layer=0
iterations=0
new_accuracy=1
accuracies=[]
weights=[]
while(new_accuracy > 0):
    reload(cl)
    new_weights, new_accuracy = cl.clustering(model,target_layer,layer1,layer2,layer3,X_test,Y_test,X_train,Y_train, weight_bias_size, relu_size);
    iterations = iterations + 1
    print("Accuracy of iteration "+str(iterations)+" for the target layer "+str(target_layer)+" is: "+str(new_accuracy))
    model.set_weights(new_weights)
    accuracies.append(new_accuracy)
    weights.append(new_weights)
    if new_accuracy < accuracy[1]-0.03:
        break

In [ ]:
max_value = max(accuracies)
if max_value >= accuracy[1]-0.03:
    print("here")
    max_index = accuracies.index(max_value)
    model.set_weights(weights[max_index])
else:
    model.set_weights(w1)
previous_weights=model.get_weights()

### weight clustering for the second layer

In [ ]:
target_layer=2
iterations=0
new_accuracy=1
accuracies_1=[]
weights_1=[]
while(new_accuracy > 0):
    reload(cl)
    new_weights, new_accuracy = cl.clustering(model,target_layer,layer1,layer2,layer3,X_test,Y_test,X_train,Y_train, weight_bias_size, relu_size);
    iterations = iterations + 1
    print("Accuracy of iteration "+str(iterations)+" for the target layer "+str(target_layer)+" is: "+str(new_accuracy))
    model.set_weights(new_weights)
    accuracies_1.append(new_accuracy)
    weights_1.append(new_weights)
    if new_accuracy < accuracy[1]-0.03:
        break
    

In [ ]:
accuracies_1

In [ ]:
new_weights

In [ ]:
max_value = max(accuracies_1)
if max_value >= accuracy[1]-0.03:
    print("here")
    max_index = accuracies_1.index(max_value)
    model.set_weights(weights_1[max_index])
else:
    model.set_weights(previous_weights)

In [ ]:
model.get_weights()

In [ ]:
model.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
model_save_quantized_weights(model, "test_weights")
model.evaluate(X_test,Y_test)

In [ ]:
#GZ
print("copy top.v, sim.Xtest, and sim.Ytest to the synopsys project ")
from qkeras.utils import model_save_quantized_weights
model_save_quantized_weights(model, "test_weights")

def aptype_to_size(ap):
    if "ap_int" in ap:
        s=ap.split('<')[1].split('>')[0]
        i=s
    else:
        s,i=ap.split('<')[1].split('>')[0].split(',')
    return (int(s),int(i))

#get weights
print(model.get_weights())
#scale weights
allweights=[t*2**(weight_bias_size[j//2][j%2][0]-weight_bias_size[j//2][j%2][1]) for j,t in enumerate(model.get_weights())]
#scale=[2**(weight_bias_size[j//2][j%2][0]-weight_bias_size[j//2][j%2][1]) for j,_ in enumerate(model.get_weights())]
#print(scale)
print(allweights)
#cast weights to integer & transpose
allweightsT=[wl.T.astype(int).tolist() for wl in allweights]
print(allweightsT)
weight_list=allweightsT[0::2]
bias_list=allweightsT[1::2]
print(weight_list)
print(bias_list)

#set params
import hls4ml
config = hls4ml.utils.config_from_keras_model(model, granularity='name')

last_layer="linear"
input_size = (4,0)
relu_size= aptype_to_size(config['LayerName']['relu1']['Precision']['result'])


weight_bias_size= [
    [aptype_to_size(config['LayerName']['fc1']['Precision']['weight']),aptype_to_size(config['LayerName']['fc1']['Precision']['bias'])],
    [aptype_to_size(config['LayerName']['output']['Precision']['weight']),aptype_to_size(config['LayerName']['output']['Precision']['bias'])]
]  
sum_relu_size=[
    [(16,6),relu_size],
    [(16,6),(32,6)]
]
print(weight_bias_size)
print(sum_relu_size)


f=open("top_clustered_custom.v","w")

import write_mlp_mergemult_ps as wv
wv.write_mlp_verilog(f, input_size, bias_list, weight_list, weight_bias_size, sum_relu_size,last_layer)
f.close()

f=open("sim.Xtest","w")
np.savetxt(f,(X_test*2**input_size[0]).astype(int),fmt='%d',delimiter=' ')
f.close()

from joblib import dump
dump(np.argmax(Y_test,axis=1), "sim.Ytest")

print("copy top.v, sim.Xtest, and sim.Ytest to the synopsys project ")


### Weight Clustering using tf

In [ ]:
model_cl = Sequential()
model_cl.add(Dense(layer2, input_shape=(layer1,), name='fc1', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model_cl.add(Activation(activation='relu', name='relu1'))
model_cl.add(Dense(layer3, name='output', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model_cl.add(Activation(activation='softmax', name='softmax'))

model_cl.set_weights(w1)


In [ ]:
import tensorflow_model_optimization as tfmot

cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 50,
  'cluster_centroids_init': CentroidInitialization.LINEAR
}

# Cluster a whole model
clustered_model = cluster_weights(model_cl, **clustering_params)

# Use smaller learning rate for fine-tuning clustered model
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)

clustered_model.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=opt,
  metrics=['accuracy'])

clustered_model.summary()


In [ ]:
new_Y_train = np.argmax(Y_train,axis=1)

In [ ]:
clustered_model.fit(
  X_train,
  new_Y_train,
  batch_size=1,
  epochs=1,
  validation_split=0.2)


In [ ]:
final_clustered_model = tfmot.clustering.keras.strip_clustering(clustered_model)
clustered_model_weights=final_clustered_model.get_weights()

In [ ]:
model_clustered_quantized = Sequential()
    # Add an input layer 

#
   
model_clustered_quantized.add(QDense(layer2, input_shape=(layer1,), name='fc1', kernel_quantizer=quantized_bits(weight_bias_size[0][0][0],0,alpha=1,use_stochastic_rounding=True),bias_quantizer=quantized_bits(weight_bias_size[0][1][0],0,alpha=1),
                 kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)   ))
model_clustered_quantized.add(QActivation(activation=quantized_relu(relu_size[0],relu_size[1],use_stochastic_rounding=False), name='relu1'))
model_clustered_quantized.add(QDense(layer3, name='output',
                 kernel_quantizer=quantized_bits(weight_bias_size[1][0][0],0,alpha=1,use_stochastic_rounding=True), bias_quantizer=quantized_bits(weight_bias_size[1][1][0],0,alpha=1),
                 kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001 ) ))
model_clustered_quantized.add(Activation(activation='softmax', name='softmax'))

model_clustered_quantized.set_weights(clustered_model_weights)


In [ ]:
model_save_quantized_weights(model_clustered_quantized, "test_weights_tf_cluster")
model_clustered_quantized.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
model_clustered_quantized.evaluate(X_test,Y_test)

In [ ]:
#GZ
print("copy top.v, sim.Xtest, and sim.Ytest to the synopsys project ")
from qkeras.utils import model_save_quantized_weights
model_save_quantized_weights(model_clustered_quantized, "test_weights")

def aptype_to_size(ap):
    if "ap_int" in ap:
        s=ap.split('<')[1].split('>')[0]
        i=s
    else:
        s,i=ap.split('<')[1].split('>')[0].split(',')
    return (int(s),int(i))

#get weights
print(model_clustered_quantized.get_weights())
#scale weights
allweights=[t*2**(weight_bias_size[j//2][j%2][0]-weight_bias_size[j//2][j%2][1]) for j,t in enumerate(model_clustered_quantized.get_weights())]
#scale=[2**(weight_bias_size[j//2][j%2][0]-weight_bias_size[j//2][j%2][1]) for j,_ in enumerate(model.get_weights())]
#print(scale)
print(allweights)
#cast weights to integer & transpose
allweightsT=[wl.T.astype(int).tolist() for wl in allweights]
print(allweightsT)
weight_list=allweightsT[0::2]
bias_list=allweightsT[1::2]
print(weight_list)
print(bias_list)

#set params
import hls4ml
config = hls4ml.utils.config_from_keras_model(model_clustered_quantized, granularity='name')

last_layer="linear"
input_size = (4,0)
relu_size= aptype_to_size(config['LayerName']['relu1']['Precision']['result'])


weight_bias_size= [
    [aptype_to_size(config['LayerName']['fc1']['Precision']['weight']),aptype_to_size(config['LayerName']['fc1']['Precision']['bias'])],
    [aptype_to_size(config['LayerName']['output']['Precision']['weight']),aptype_to_size(config['LayerName']['output']['Precision']['bias'])]
]  
sum_relu_size=[
    [(16,6),relu_size],
    [(16,6),(32,6)]
]
print(weight_bias_size)
print(sum_relu_size)


f=open("top_clustered_keras.v","w")

import write_mlp_mergemult_ps as wv
wv.write_mlp_verilog(f, input_size, bias_list, weight_list, weight_bias_size, sum_relu_size,last_layer)
f.close()

f=open("sim.Xtest","w")
np.savetxt(f,(X_test*2**input_size[0]).astype(int),fmt='%d',delimiter=' ')
f.close()

from joblib import dump
dump(np.argmax(Y_test,axis=1), "sim.Ytest")

print("copy top.v, sim.Xtest, and sim.Ytest to the synopsys project ")
